# LSTM WORKING 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test', 'train.csv']


In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR
from sklearn.metrics import mean_absolute_error

In [3]:
train = pd.read_csv('../input/train.csv', dtype={'acoustic_data': np.float32, 'time_to_failure': np.float32}).values

In [4]:
train [0:150000, 0 ] .mean(axis=0)

4.8841133

In [5]:
# pandas doesn't show us all the decimals
pd.options.display.precision = 15

In [6]:
rows = 150_000
segments = int(np.floor(train.shape[0] / rows))
print('train.shape',train.shape)
segments


train.shape (629145480, 2)


4194

In [ ]:
X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])
y_train.shape



segments *rows+rows

In [ ]:
# Create a training file with simple derived features
 
# total train.shape (629145480, 2)

rows = 150_000
segments = int(np.floor(train.shape[0] / rows))                         # 4194

X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min'])             # 4194,4
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])                      # 4194,1

for segment in tqdm(range(segments)):                          # for loop from 1 to 4194 segment value
    seg = train.iloc[segment*rows:segment*rows+rows]           # seg.shape (150000,2)  i-e first iteration (0-150000) then (150000-300000) then (300000-4500000) ...... (629100000-629250000)
    x = seg['acoustic_data'].values                            # x.shape (150000,)
    y = seg['time_to_failure'].values[-1]                      # i-e last time_to_failure value of every 150000 length segment. 
    
    y_train.loc[segment, 'time_to_failure'] = y       # 4194,1      ie go on from (1,1) to (4194,1)
    
    X_train.loc[segment, 'ave'] = x.mean()         # 4194,4          ie go on from (1,4) to (4194,4)
    X_train.loc[segment, 'std'] = x.std()          # 4194,4          ie go on from (1,4) to (4194,4)
    X_train.loc[segment, 'max'] = x.max()          # 4194,4          ie go on from (1,4) to (4194,4)
    X_train.loc[segment, 'min'] = x.min()          # 4194,4          ie go on from (1,4) to (4194,4)

# **Extracting Features**

In [7]:
def extract_features(z):
     return np.c_[z.mean(axis=0), 
                  z.std(axis=0),
                  z.max(axis=0),
                  z.min(axis=0),
                #  np.transpose(np.percentile(np.abs(z), q=[0, 50, 75, 100], axis=0)) .reshape(1,4)
                 ]

In [8]:
def create_X(x, last_index=None, n_steps=150, step_length=1000):
    if last_index == None:
        last_index=len(x)
       
    assert last_index - n_steps * step_length >= 0

    # Reshaping and approximate standardization with mean 5 and std 3.
    # ORIGINAL: I changed this becuase I got an No OpKernel was registered to support Op 'CudnnRNN' error
    #temp = (x[(last_index - n_steps * step_length):last_index].reshape(n_steps, -1) - 5 ) / 3
    # MY CHANGE: This doesn't fix things, I get the same errors
    temp = (x[(int(last_index) - n_steps * step_length):int(last_index)].reshape(n_steps,step_length ).astype(np.float32) - 5 ) / 3
    
    # Extracts features of sequences of full length 1000, of the last 100 values and finally also 
    # of the last 10 observations. 
    return np.c_[extract_features(temp),
                 extract_features(temp [ -n_steps // 10:]),
                 extract_features(temp [ -n_steps // 100:])]



# Creating Training Data

In [9]:
# Query "create_X" to figure out the number of features
n_features = create_X(train [0:150000,0]).shape[1]
print("Our RNN is based on %i features"% n_features)     # 18 features each row of segment ie 150x18 features of 150000 chunk input

n_steps=150
step_length=1000
maxsize=train .shape[0]
seg = int(np.floor(maxsize / (n_steps*step_length))) 
batch_size = seg-1   # (4193,) 
xx=350


##############################################################################################
rows_initialize = np.zeros((seg), dtype=float)
print(rows_initialize.shape)

for seg1 in tqdm(range(1,seg)) :      # for loop from 1 to 4194 segment value
    rows_initialize [seg1] = seg1 * (n_steps*step_length) 

rows=np.delete(rows_initialize,0)    # (4193,)

print(rows.shape)

########################################################################################
batch_size=batch_size-xx    # training data
#batch_size=xx              # validation data
split_point=xx
second_earthquake = rows[xx]



##########################################################################################

if batch_size < 1000  :    # validation set 
               rows_1 = rows[:split_point+1]    #  0:350 
        
if batch_size > 1000 :   # training set
               rows_1 = rows[split_point+1 :]    # (351,) ie 351:4193    
            

       
    # Initialize feature matrices and targets
samples_tr= np.zeros((rows_1.shape[0], step_length, n_features), dtype=float)   # (16, 150, 18)  for validation (350, 1, 24)  for training ( 3843, 1, 24) 
targets_tr = np.zeros(rows_1.shape[0], )    # (16,)  for validation (350)    for training ( 3843)
        
for j, row in enumerate(rows_1):             # 16 for validation (350)    for training ( 3843)
    samples_tr[j] = create_X(train[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
    targets_tr[j] = train[int(row - 1), 1]         
    
    
################################################################################################

print('samples_tr shape', samples_tr.shape)
print('targets_tr shape', targets_tr.shape)




100%|██████████| 4193/4193 [00:00<00:00, 1529279.71it/s]

Our RNN is based on 12 features
(4194,)
(4193,)


samples_tr shape (3842, 1000, 12)
targets_tr shape (3842,)


# Creating Validation data

In [10]:
#batch_size=batch_size-xx    # training data
batch_size=xx              # validation data
split_point=xx
second_earthquake = rows[xx]

##########################################################################################

if batch_size < 1000  :    # validation set 
               rows_1 = rows[:split_point+1]    #  0:350 
        
if batch_size > 1000 :   # training set
               rows_1 = rows[split_point+1 :]    # (351,) ie 351:4193    
            
       
    # Initialize feature matrices and targets
samples_vd= np.zeros((rows_1.shape[0], step_length, n_features), dtype=float)   #   for validation (350, 1, 24) 
targets_vd = np.zeros(rows_1.shape[0], )    #  for validation (350)    
        
for j, row in enumerate(rows_1):             # 16 for validation (350)    for training ( 3843)
    samples_vd[j] = create_X(train[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
    targets_vd[j] = train[int(row - 1), 1]  
    
    
print('samples_vd shape', samples_vd.shape)
print('targets_vd shape',targets_vd.shape)  
#print('rows_1 shape',rows_1.shape[0])
    

samples_vd shape (351, 1000, 12)
targets_vd shape (351,)


In [11]:
# Query "create_X" to figure out the number of features
n_features = create_X(train [0:150000,0]).shape[1]
print("Our RNN is based on %i features"% n_features)     # 18 features each row of segment ie 150x18 features of 150000 chunk input

n_steps=150000
step_length=1

# initialization for rows
maxsize=train .shape[0]
seg = int(np.floor(maxsize / (n_steps*step_length)))    #4194
segment = (n_steps*step_length)* seg    # (150000* 4194) = 629100000
rows = np.zeros(seg, )    # (4194,)          
          
for seg1 in tqdm(range(1,seg)) :      # for loop from 1 to 4194 segment value
    rows [seg1] = seg1 * (n_steps*step_length) 

rows=np.delete(rows,0)    # (4193,) 

# The generator endlessly selects "batch_size" ending positions of sub-time series. For each ending position,
# the "time_to_failure" serves as target, while the features are created by the function "create_X".
def generator(data, min_index=0, max_index=None, batch_size=16, n_steps=150000, split_point=350, step_length=1):    
    if max_index is None:
        max_index = len(data) - 1     # data = float_data  len= 629145480-1
     
   # while True:
       # # Pick indices of ending positions
 #####       rows_1 = np.random.randint(min_index + n_steps * step_length, max_index, size=batch_size)    # produce 16 random number bw min index and max index
              
        if batch_size < 1000  :    # validation set 
               rows_1 = rows[:split_point+1]    #  0:350 
        
        if batch_size > 1000 :   # training set
               rows_1 = rows[split_point+1 :]    # (351,) ie 351:4193    
            

       
    # Initialize feature matrices and targets
        samples = np.zeros((batch_size, step_length, n_features), dtype=float)   # (16, 150, 18)  for validation (350, 1, 24)  for training ( 3843, 1, 24) 
        targets = np.zeros(batch_size, )    # (16,)  for validation (350)    for training ( 3843)
        
        for j, row in enumerate(rows_1):             # 16 for validation (350)    for training ( 3843)
            samples[j] = create_X(data[:, 0], last_index=row, n_steps=n_steps, step_length=step_length)
            targets[j] = data[int(row - 1), 1]
        
    return samples, targets
        
       
batch_size = seg-1   # (4193,) 
###### batch_size = 32

# Position of second (of 16) earthquake. Used to have a clean split
# between train and validation
xx=350
second_earthquake = rows[xx]

#float_data[second_earthquake, 1] # time failure corresponding to this second earthquake indices

# Initialize generators
#train_gen = generator(float_data, batch_size=batch_size) # Use this for better score

train_gen = generator(train , batch_size=batch_size-xx, split_point=xx, min_index=second_earthquake + 1)     # batch_size-xx =  3843
#valid_gen = generator(train , batch_size=xx, max_index=second_earthquake)                    # xx =  350

######train_gen = generator(float_data, batch_size=batch_size, split_point=xx, min_index=second_earthquake + 1) 
######valid_gen = generator(float_data, batch_size=batch_size, max_index=second_earthquake)


#print(samples.shape)
#print(targets.shape) 


100%|██████████| 4193/4193 [00:00<00:00, 1047015.34it/s]

Our RNN is based on 12 features


# Define the model

In [12]:
from keras.models import Sequential
from keras.layers import Dense, CuDNNGRU, SimpleRNN, LSTM
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

cb = [ModelCheckpoint("model.hdf5", save_best_only=True, period=3)]

model = Sequential()
model.add(LSTM(48,dropout=0.2, recurrent_dropout=0.2, input_shape=(None, n_features), return_sequences=True))
model.add(LSTM(20,dropout=0.2, recurrent_dropout=0.2, return_sequences=False))
####model.add(CuDNNGRU(48, input_shape=(None, n_features)))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))

model.summary()

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 48)          11712     
_________________________________________________________________
lstm_2 (LSTM)                (None, 20)                5520      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               2100      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 19,433
Trainable params: 19,433
Non-trainable params: 0
_________________________________________________________________


# Compile and fit model

In [15]:
import keras
from keras.optimizers import RMSprop
opt = keras.optimizers.adam(lr=.01)

model.compile(loss="mae",
              optimizer=opt, metrics=['mean_absolute_error'])
             # metrics=['accuracy'])


batch_size = 128 # mini-batch with 32 examples
epochs = 10
history = model.fit(
    samples_tr, targets_tr,
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_data=(samples_vd  ,targets_vd ))

Train on 3842 samples, validate on 351 samples
Epoch 1/10
3842/3842 [==============================] - 78s 20ms/step - loss: 2.7765 - mean_absolute_error: 2.7765 - val_loss: 3.0701 - val_mean_absolute_error: 3.0701
Epoch 2/10
3842/3842 [==============================] - 77s 20ms/step - loss: 2.8524 - mean_absolute_error: 2.8524 - val_loss: 2.6600 - val_mean_absolute_error: 2.6600
Epoch 3/10
3842/3842 [==============================] - 77s 20ms/step - loss: 2.5537 - mean_absolute_error: 2.5537 - val_loss: 2.6698 - val_mean_absolute_error: 2.6698
Epoch 4/10
3842/3842 [==============================] - 76s 20ms/step - loss: 2.6146 - mean_absolute_error: 2.6146 - val_loss: 2.6289 - val_mean_absolute_error: 2.6289
Epoch 5/10
3842/3842 [==============================] - 76s 20ms/step - loss: 2.5442 - mean_absolute_error: 2.5442 - val_loss: 2.6386 - val_mean_absolute_error: 2.6386
Epoch 6/10
3842/3842 [==============================] - 76s 20ms/step - loss: 2.6095 - mean_absolute_error: 2.609

In [ ]:
model.compile(optimizer=adam(lr=0.0005), loss="mae", metrics=['accuracy'])

history = model.fit_generator(train_gen,
                              steps_per_epoch=1000,
                              epochs=30,
                              verbose=1,
                              callbacks=cb,
                              validation_data=valid_gen,
                              validation_steps=351)

# Load submission file


In [16]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})

## Prepare submission data
Load each test data, create the feature matrix, get numeric prediction

In [17]:
for i, seg_id in enumerate(tqdm(submission.index)):
  #  print(i)
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    x = seg['acoustic_data'].values
    submission.time_to_failure[i] = model.predict(np.expand_dims(create_X(x), 0))

submission.head()

100%|██████████| 2624/2624 [21:02<00:00,  2.06it/s]


,time_to_failure
seg_id,
seg_00030f,5.253851890563965
seg_0012b5,5.233308792114258
seg_00184e,5.474508285522461
seg_003339,8.704728126525879
seg_0042cc,5.388502120971680


## Save submission file

In [18]:
submission.to_csv('submission.csv')

# OTHER TECHNIQUES

In [ ]:
x.mean()  

In [ ]:
X_train

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
X_train_scaled

In [ ]:
y_train.values

In [ ]:
y_train.values.flatten()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

## Train the  Model
model = RandomForestRegressor(n_estimators=200)
model.fit(X_train_scaled, y_train.values.flatten())      # .fit used for training
y_pred = model.predict(X_train_scaled)

In [ ]:
# number support vector regressor

# svm = NuSVR()
# svm.fit(X_train_scaled, y_train.values.flatten())
# y_pred = svm.predict(X_train_scaled)

In [ ]:
y_pred.shape

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_train.values.flatten(), y_pred)
plt.xlim(0, 20)
plt.ylim(0, 20)
plt.xlabel('actual', fontsize=12)
plt.ylabel('predicted', fontsize=12)
plt.plot([(0, 0), (20, 20)], [(0, 0), (20, 20)])
plt.show()

In [ ]:
score = mean_absolute_error(y_train.values.flatten(), y_pred)
print(f'Score: {score:0.3f}')

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')

In [ ]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)


In [ ]:
for seg_id in X_test.index:
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()

In [ ]:
X_test_scaled = scaler.transform(X_test)
submission['time_to_failure'] = svm.predict(X_test_scaled)
submission.to_csv('submission.csv')